In [94]:
## INDUSTRY BY COUNTY DATA - Alanis Perez

In [96]:
import pandas as pd
import glob
import os

# Ignore all warnings
import warnings
warnings.filterwarnings("ignore")

In [98]:
# Define paths (input & output)
input_directory = 'data/OG_county'
output_directory = 'data/Industry_combined_data'

# Function to combine industry into categories
def industry_combine(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Find the index of "Total, All Industries" (SKIP ALL ROWS PRIOR THIS LINE!)
    start_index = None
    for i, line in enumerate(lines):
        if "Total, All Industries" in line:
            start_index = i + 1  # Start AFTER "Total, All Industries"
            break

    if start_index is not None:
        data = []
        for line in lines[start_index:]:
            stripped_line = line.strip()
            leading_spaces = len(line) - len(stripped_line)
            indentation_level = leading_spaces // 2  # There are 2 spaces per indentation
            
            # Append the line and its indentation level to the data list
            data.append((stripped_line, indentation_level))

        # Create a DataFrame from the data
        df = pd.DataFrame(data, columns=['Industry', 'Indentation Level'])

        # Filter the DataFrame to include only rows with indentation levels of 3 or greater
        df_filtered = df[df['Indentation Level'] >= 3] # Anything below 3 indents will not be a good representation of the category

        # Create a new DataFrame to hold combined categories
        combined_data = []
        current_main_category = None
        current_subcategories = []

        for index, row in df_filtered.iterrows():
            industry = row['Industry']
            indentation_level = row['Indentation Level']

            if indentation_level == 3: # This will be the main category
                if current_main_category is not None:
                    combined_data.append((current_main_category, ', '.join(current_subcategories)))

                current_main_category = industry # Start the new main category
                current_subcategories = []  # Reset subcategories list

            elif indentation_level == 4: # This will be the subcategory
                current_subcategories.append(industry)

        if current_main_category is not None:
            combined_data.append((current_main_category, ', '.join(current_subcategories)))

        # Create new DF from the combined data
        combined_df = pd.DataFrame(combined_data, columns=['Main Category', 'Subcategories'])

        return combined_df
    else:
        print("No main categories found.")
        return pd.DataFrame()  # In case any data frames come back empty

# Process each CSV file (county) in the input directory
for filename in os.listdir(input_directory):
    if filename.endswith('.csv'):
        file_path = os.path.join(input_directory, filename)
        combined_df = industry_combine(file_path)

        # Save the combined DataFrame to a new CSV file in the output directory
        output_file_path = os.path.join(output_directory, f'combined_{filename}')
        combined_df.to_csv(output_file_path, index=False)

print("Processing complete. Combined files saved in 'Industry_combined_data' directory.")

Processing complete. Combined files saved in 'Industry_combined_data' directory.


In [100]:
# Define paths (input & output)
input_directory = 'data/Industry_combined_data'
output_directory = 'data/Industry_transformed_data'

# Function to transform/split data from categorization done using the previous function
def transform_data(file_path):
    # Load the combined data
    df = pd.read_csv(file_path)

    # Split the 'Main Category' column
    categories_split = df['Main Category'].str.split(',', expand=True)

    # Create new column names (one for each year from 2010 to 2024)
    year_columns = [f"{year}_Average" for year in range(2010, 2025)]

    # Create data frame with the proper columns for our data
    transformed_df = pd.DataFrame(columns=["Main Category"] + year_columns)

    # Iterate through each row in the split categories
    for index, row in categories_split.iterrows():
        # The first value is the Main Category
        main_category = row[0]
        
        # The remaining values are the averages
        averages = row[1:].tolist()
        
        # Fill the remaining values with NaN in case there are fewer than 15
        while len(averages) < 15:
            averages.append(float('nan'))  # Append NaN in case of gaps/blanks
        
        # Create a new row for the transformed DataFrame
        transformed_df.loc[index] = [main_category] + averages[:15]  # Takes the first 15 values

    return transformed_df

# Process each CSV file (combined_data) in the input directory
for filename in os.listdir(input_directory):
    if filename.endswith('.csv'):
        file_path = os.path.join(input_directory, filename)
        transformed_df = transform_data(file_path)

        # Save the transformed DataFrame to a new CSV file in the output directory
        output_file_path = os.path.join(output_directory, f'transformed_{filename}')
        transformed_df.to_csv(output_file_path, index=False)

print("Transformation complete. Transformed files saved in 'Industry_transformed_data' directory.")

Transformation complete. Transformed files saved in 'Industry_transformed_data' directory.


In [116]:
# CREATE DATABASE TO HOLD ALL THIS DATA !
# Create a list to hold files for all 58 counties
dataframes = []

# Loop through all CSV files in the directory
for county in glob.glob("data/Industry_transformed_data/*.csv"):
    
    # Read the CSV file
    county_df = pd.read_csv(county)
    
    # Extract county name from the filename, assign county ID to be name of county
    county_id = county.split("/")[-1].replace(".csv", "").replace("Industry_transformed_data\\transformed_combined_", "").replace("_county", "")
    county_df['County_ID'] = county_id
    
    # Append the DataFrame to the list
    dataframes.append(county_df)

# Concatenate all DataFrames into one
combined_df = pd.concat(dataframes, ignore_index=True)

In [128]:
# combined_df.head(50)
industry_df = combined_df.set_index("County_ID")
industry_df.head(50)

,Main Category,2010_Average,2011_Average,2012_Average,2013_Average,2014_Average,2015_Average,2016_Average,2017_Average,2018_Average,2019_Average,2020_Average,2021_Average,2022_Average,2023_Average,2024_Average
County_ID,,,,,,,,,,,,,,,,
Alameda,Mining,"Logging and Construction""",30308.33,30833.33,33283.33,35550.00,37600.00,40875.00,43041.67,45925.00,49041.67,49591.67,46650.00,48675.00,48416.67,47391.67
Alameda,Manufacturing,60150.0,62141.67,62325.0,64641.67,67508.33,72466.67,75008.33,79941.67,84716.67,85033.33,83658.33,91841.67,98283.33,97750.00,63353.33
Alameda,Trade,"Transportation & Utilities""",117800.0,119150.0,123250.00,127658.33,131300.00,135683.33,137900.00,138291.67,139733.33,138816.67,129366.67,134041.67,137208.33,137100.00
Alameda,Information,14333.33,14166.67,14491.67,14066.67,14683.33,16641.67,18233.33,18791.67,19966.67,20533.33,19900.00,18791.67,19141.67,18616.67,14348.33
Alameda,Financial Activities,23333.33,24491.67,26066.67,28033.33,29058.33,29283.33,30333.33,28733.33,28008.33,28350.00,27033.33,27091.67,27075.00,27283.33,26196.67
Alameda,Professional and Business Services,108133.33,111025.0,117258.33,118350.00,120350.00,125666.67,128733.33,130383.33,134025.00,136625.00,128908.33,134358.33,137841.67,135008.33,115023.33
Alameda,Private Education and Health Services,105016.67,104575.0,108283.33,111025.00,111625.00,114491.67,117558.33,122150.00,123591.67,125958.33,121783.33,125116.67,128508.33,135825.00,108105.00
Alameda,Leisure and Hospitality,54466.67,55966.67,58333.33,61883.33,65766.67,68366.67,70916.67,74116.67,76425.00,78000.00,53025.00,57866.67,68616.67,71141.67,59283.33
Alameda,Other Services,23191.67,23341.67,23991.67,24808.33,24958.33,25450.00,26225.00,27191.67,27458.33,27491.67,22475.00,23800.00,25775.00,27191.67,24058.33


In [ ]:
# Export to JSON
industry_df.to_json("data/industry_county_data.json", orient="records")